Information retrieval - информационный поиск.

Вот есть у нас, положим, коллекция песен.

In [ ]:
song_collection = ["Эта песня простая",
                   "Песня про зайцев",
                   "Я в моменте",
                   "Я это ты, ты это я",
                   "Индийский чай",
                   "В синем море, в белой пене",
                   "Ягода малинка",
                   "Ты горишь как огонь",
                   "Никаких больше вечеринок",
                   "Покинула чат"]

Мы хотим в это коллекции найти песню про зайцев. Это будет изи. Но сначала разберемся, что на входе и что на выходе у нашего поисковика.

Input (вход): текстовая строка, поисковый запрос ("песня про зайцев") - он же query (квИри).

Output (выход): порядковый номер песни в коллекции. Напоминаю, что в списках нумерация элементов списка начинается с 0.

In [ ]:
query_1 = 'Песня про зайцев'

In [ ]:
# Изи-пизи-лемон-сквизи
song_collection.index(query_1)

1

Но не все пользователи такие послушные, что название песни печатают точно так, как оно в коллекции...

In [ ]:
query_2 = 'песня, про зайцев'
song_collection.index(query_2)

ValueError: ignored

Нам нужен препроцессинг (предобработка) текстов как запроса, так и названий песен в коллекции. Сделаем функцию, которая получает на входе сырой текст, а на выходе возвращает обработанный текст.

In [ ]:
def preprocessor(text):
    text = text.lower()
    text = text.replace(',', '')
    return text

Обработаем сначала нашу коллекцию.

In [ ]:
preprocessed_collection = []
for sc in song_collection:
    preprocessed_collection.append(preprocessor(sc))

In [ ]:
preprocessed_collection

['эта песня простая',
 'песня про зайцев',
 'я в моменте',
 'я это ты ты это я',
 'индийский чай',
 'в синем море в белой пене',
 'ягода малинка',
 'ты горишь как огонь',
 'никаких больше вечеринок',
 'покинула чат']

Теперь обработаем query.

In [ ]:
preprocessed_query_2 = preprocessor(query_2)

In [ ]:
preprocessed_query_2

'песня про зайцев'

In [ ]:
preprocessed_collection.index(preprocessed_query_2)

1

Чуть сложнее, но все-таки довольно просто будет найти песню про море и песню про чай.

In [ ]:
query_3 = 'море'
query_4 = 'чай'

In [ ]:
for pc in preprocessed_collection:
    if query_3 in pc:
        print("Песня про море: ", preprocessed_collection.index(pc))
    if query_4 in pc:
        print("Песня про чай: ", preprocessed_collection.index(pc))

Песня про чай:  4
Песня про море:  5


А теперь попробуем найти песню про вечеринки. В нашей коллекции это слово встречается только в форме "вечеринок". Позовем на помощь лемматизатор - программу, которая возвращает начальную форму слова. Для глагола это будет инфинитив, для существительного - единственное число (если есть), именительный падеж, и т.д. Будем использовать библиотеку Spacy. Сначала нужно установить модель для русского языка. Грузится долговато - терпение, только терпение!

In [ ]:
!python -m spacy download ru_core_news_lg

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('ru_core_news_lg')

Создадим функцию, которая будет преобразовывать текст в набор лемм (начальных форм слов).

In [ ]:
def lemmatizer(text):
    spacy_text = nlp(text)
    lemmas = ''
    for st in spacy_text:
        lemmas += st.lemma_+' '
    return lemmas

Вот такой текст получается в итоге.

In [ ]:
lemmatizer(song_collection[0])

'этот песня простой '

Лемматизируем коллекцию.

In [ ]:
lemmatized_songs = []
for sc in song_collection:
    lemmatized_songs.append(lemmatizer(sc))

In [ ]:
lemmatized_songs

['этот песня простой ',
 'песня про заяц ',
 'я в момент ',
 'я это ты , ты это я ',
 'индийский чай ',
 'в синий море , в белый пена ',
 'ягода малинка ',
 'ты гореть как огонь ',
 'никакой больше вечеринка ',
 'покинуть чат ']

In [ ]:
query_5 = 'вечеринка'

In [ ]:
for pc in lemmatized_songs:
    if query_5 in pc:
        print("Песня про вечеринку: ", lemmatized_songs.index(pc))

Песня про вечеринку:  8


А теперь усложним все запредельно. Поищем-ка песни про любовь! Стоит отметить, что любовь - это очень сложное понятие. Поэтому результатом будет не какая-то конкретная пенся, а весь список песен, сортированный от самой "любовной" песни до самой "нелюбовной". Будем использовать семантические вектора, которым будут посвящены еще несколько постов в этом месяце. А пока просто посмотрим на них как на какую-то магию ✨

In [ ]:
def semantizer(text):
    spacy_text = nlp(text)
    return spacy_text.vector

In [ ]:
semantized_songs = []
for sc in song_collection:
    semantized_songs.append(semantizer(sc))

Семантический вектор каждой песни теперь - это набор из 300 чисел.

In [ ]:
semantized_songs[0].shape

(300,)

Теперь семантизируем любовь.

In [ ]:
love_vector = semantizer('любовь')

В ноутбуке про векторизацию я рассказывала, как отличать смешные твиты от несмешных через расстояние между векторами твитов. https://colab.research.google.com/drive/1B5_WI_zkth-lFo-HQpSGlOXz0TFgTP42?usp=sharing

В нашей задаче информационного поиска песен про любовь будем делать это через меру под названием "косинусная близость" - о ней тоже будет отдельный пост.

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
def similarity(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [ ]:
print(similarity(semantized_songs[0], love_vector))

0.36921203


А теперь оценим, какие, по мнению ИИ, песни больше поют о любви, а какие - меньше.

In [ ]:
for sems in semantized_songs:
    print(similarity(sems, love_vector))

0.36921203
0.24015385
0.15716562
0.42140687
0.12162422
0.10965233
-0.089701205
0.21724573
0.21324694
-0.066468276


Победитель рейтинга: "Я это ты, ты это я" (0.42140687) - с этим трудно не согласиться. На втором месте "Эта песня простая" (0.36921203) - хм, ну возможно.. "Ягода малинка" и "Покинула чат" - антилидеры нашего рейтинга. Мне кажется, тут есть над чем поработать. Но, опять же, это же только по заголовкам. В общем, инфомрационный поиск все еще очень далек от совершенства. Следовательно, и нам с вами работа найдется 😀